In [77]:
import numpy as np

kontury = "Kontury_eksport_dz.txt"

data = np.array([])
with open(kontury, mode='r') as file:
    for line in file:
        data = np.append(data, line.strip())

codes = []
i = 0
while (i+2) < len(data):
    code = data[i]
    num_of_lines = int(data[i+2])
    codes.append(code)
    i += num_of_lines + 4

possible_ofus = set(['B', 'Ba', 'Bi', 'Bp', 'Bz', 'K', 'dr', 'Tk', 'Ti', 'Tp', 'Wm', 'Wp', 'Ws', 'Tr', 'Ls', 'Lz', 'N', 'S', 'Br', 'Wsr', 'W', 'Lzr', 'R', 'Ps', 'Ł'])
unclassified = set(['B', 'Ba', 'Bi', 'Bp', 'Bz', 'K', 'dr', 'Tk', 'Ti', 'Tp', 'Wm', 'Wp', 'Ws', 'Tr', 'N'])
r_classes = set(['I', 'II', 'IIIa', 'IIIb', 'IVa', 'IVb', 'V', 'VI', 'VIz'])
lpslslz_classes = set(['I', 'II', 'III', 'IV', 'V', 'VI'])

errors = []

In [78]:
def add_error(code, error_code, errors_array = errors):
    errors_array.append({'code': code, 'error_code': error_code})

def check_obreb(code, dash):
    for char in code[:dash]:
        if not char.isdigit():
            add_error(code, 'ERR03')
            return 1

def check_dzialka(code, dash, slash):
    for char in code[dash+1:slash]:
        if not char.isdigit():
            add_error(code, 'ERR04')
            return 1

def check_ofu(code, slash_index):
    if code[slash_index+1:slash_index+4] in possible_ofus:
        return code[slash_index+1:slash_index+4]
    elif code[slash_index+1:slash_index+3] in possible_ofus:
        return code[slash_index+1:slash_index+3]
    elif code[slash_index+1:slash_index+2] in possible_ofus:
        return code[slash_index+1:slash_index+2]
    else:
        return None

def check_r(code, next_index):
    chars_left = len(code) - next_index

    if chars_left == 0:
        add_error(code, 'ERR07')
        return 1
    elif chars_left > 4:
        add_error(code, 'ERR08')
        return 1
    else:
        if code[next_index:] in r_classes:
            return code[next_index:]
        add_error(code, 'ERR09')
        return 1

def check_lpslslz(code, next_index):
    chars_left = len(code) - next_index

    if chars_left == 0:
        add_error(code, 'ERR07')
        return 1
    elif chars_left > 3:
        add_error(code, 'ERR08')
        return 1
    else:
        if code[next_index:] in lpslslz_classes:
            return code[next_index:]
        add_error(code, 'ERR09')
        return 1

def check_sbrwsrwlzr(code, next_index, uzytek_type):
    chars_left = len(code) - next_index
    
    if chars_left < 2:
        add_error(code, 'ERR12')
        return 1
    elif code[next_index] != '-':
        add_error(code, 'ERR10')
        return 1
    elif code[next_index + 1: next_index + 3] == 'Ps':
        if len(code) > next_index + 3:
            if code[next_index + 3:] not in lpslslz_classes:
                add_error(code, 'ERR09')
                return 1
        else:
            add_error(code, 'ERR07')
            return 1
    elif code[next_index + 1: next_index + 2] == 'Ł':
        if len(code) > next_index + 2:
            if code[next_index + 2:] not in lpslslz_classes:
                add_error(code, 'ERR09')
                return 1
        else:
            add_error(code, 'ERR07')
            return 1
    elif code[next_index + 1: next_index + 2] == 'R':
        if len(code) > next_index + 2:
            if code[next_index + 2:] not in r_classes:
                add_error(code, 'ERR09')
                return 1
        else:
            add_error(code, 'ERR07')
            return 1
    elif uzytek_type == 'W' and code[next_index + 1: next_index + 3] in ['Ls', 'Lz']:
        if len(code) > next_index + 3:
            if code[next_index + 3:] not in lpslslz_classes:
                add_error(code, 'ERR09')
                return 1
        else:
            add_error(code, 'ERR07')
            return 1
    else:
        add_error(code, 'ERR11')
        return 1

def check_unclassified(code, next_index):
    if len(code) > next_index:
        add_error(code, 'ERR06')
        return 1

def check_zgodnosc(code, uzytek_type, slash_index):
    next_index = slash_index + len(uzytek_type) + 1
    
    if uzytek_type in unclassified:
        check_unclassified(code, next_index)
    elif uzytek_type == 'R':
        check_r(code, next_index)
    elif uzytek_type in ['Ł', 'Ps', 'Ls', 'Lz']:
        check_lpslslz(code, next_index)
    elif uzytek_type in ['S', 'Br', 'Wsr', 'W', 'Lzr']:
        check_sbrwsrwlzr(code, next_index, uzytek_type)   


In [79]:
error_codes = {
    'ERR01': "Brak dasha.",
    'ERR02': "Brak slasha.",
    'ERR03': 'Niepoprawny numer obrębu.',
    'ERR04': 'Niepoprawny numer działki.',
    'ERR05': 'Nierozpoznany typ użytku.',
    'ERR06': 'Zbędne znaki dla nieklasyfikowanego użytku.',
    'ERR07': 'Brak OZK dla klasyfikowanego użytku.',
    'ERR08': 'Za dużo znaków.',
    'ERR09': 'Nierozpoznany typ klasyfikacji dla tego użytku.',
    'ERR10': 'Niepoprawne połączenie OFU-OZU.',
    'ERR11': 'Niedozwolona kombinacja OFU-OZU.',
    'ERR12': 'Za mało znaków: brak OZU i/lub OZK.'
}

for code in codes:
    dash_index = code.find('-')
    slash_index = code.find('/')

    if dash_index == -1:
        add_error(code, 'ERR01')
        continue

    if slash_index == -1:
        add_error(code, 'ERR02')
        continue

    if check_obreb(code, dash_index) == 1:
        continue

    if check_dzialka(code, dash_index, slash_index) == 1:
        continue

    uzytek_type = check_ofu(code, slash_index)
    if uzytek_type is None:
        add_error(code, 'ERR05')
        continue

    check_zgodnosc(code, uzytek_type, slash_index)

In [86]:
for i, error in enumerate(errors):
    print(f"{i+1}.    {error['code']}     {error_codes[error['error_code']]}")

1.    23-2/LsVIz     Nierozpoznany typ klasyfikacji dla tego użytku.
2.    23-3/B RIIIb     Zbędne znaki dla nieklasyfikowanego użytku.
3.    23-5/LsIVa     Nierozpoznany typ klasyfikacji dla tego użytku.
4.    23-8/RIII     Nierozpoznany typ klasyfikacji dla tego użytku.
5.    23-12/W/RIVa     Niepoprawne połączenie OFU-OZU.
6.    23-14/Lzr     Za mało znaków: brak OZU i/lub OZK.
7.    23-16/E-Wm     Nierozpoznany typ użytku.
8.    23-19/N-RVI     Zbędne znaki dla nieklasyfikowanego użytku.
9.    23-20/E     Nierozpoznany typ użytku.
10.    23-22/E-N     Nierozpoznany typ użytku.
11.    23-24/E-LsV     Nierozpoznany typ użytku.
12.    23-27/E-PsIV     Nierozpoznany typ użytku.
13.    23-28/E-W     Nierozpoznany typ użytku.
14.    23-30/E-Lzr     Nierozpoznany typ użytku.
15.    23-31/E-Lzr-RVI     Nierozpoznany typ użytku.
16.    23-33/RIV     Nierozpoznany typ klasyfikacji dla tego użytku.
17.    23-34/S-N     Niedozwolona kombinacja OFU-OZU.
18.    23-35/S     Za mało znaków: brak O

In [81]:
with open ('bledne.txt', 'w') as f:
    for error in errors:
        f.write(error['code'] + '\n')